In [1]:
from hotgrad.variable import Variable
from torch import FloatTensor
from torch.autograd import Variable as tVariable
from numpy import isclose

%load_ext autoreload
%autoreload 2

Modules to be tested:

Operands:
- add
- sub
- mul
- matmul
- pow
- mean

Layers:
- linear

Losses:
- MSE

Activations:
- ReLU
- Tanh

In [ ]:
ourModules[]